In [42]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [43]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [44]:
%autoreload 2

In [45]:
import pandas as pd
import datetime as dt

In [46]:
from datahandling import DataHandler
from preprocessing import Preprocessor

In [4]:
raw_df = DataHandler().get()

In [5]:
raw_df

,Id,CumReturnResid,CumVolume,EST_VOL,MDV_63,Volume
2010-01-04 09:45:00,BBG000MQ1SN9,0.002070,43347,0.13074,4.383275e+07,955120.0
2010-01-04 09:45:00,BBG000BBCQD7,-0.012105,229661,0.31610,4.409296e+07,2566745.0
2010-01-04 09:45:00,BBG000BBB3K1,-0.001020,62697,0.11253,7.588626e+07,812905.0
2010-01-04 09:45:00,BBG000BNMHS4,0.012179,47650,0.18568,4.413393e+07,1108030.0
2010-01-04 09:45:00,BBG000C23PB0,0.003520,239259,0.11252,4.440541e+07,3500468.0
...,...,...,...,...,...,...
2014-12-31 16:00:00,BBG000BPH459,-0.003270,16810020,0.12146,1.383197e+09,21551092.0
2014-12-31 16:00:00,BBG000GZQ728,-0.000065,9221446,0.09480,1.273231e+09,11326179.0
2014-12-31 16:00:00,BBG000CKGBP2,-0.008449,11508879,0.20535,1.589295e+09,13851283.0
2014-12-31 16:00:00,BBG000MM2P62,-0.005813,18519636,0.17343,2.258944e+09,20004654.0


In [6]:
day_sep = raw_df[["Id", "CumReturnResid"]].groupby(["Id", raw_df.index.date, raw_df.index.time>dt.time(15,30)]).last()
day_sep

CumReturnResid
Id                                                     
BBG000B9WH86           2010-01-04 False       -0.002424
                                  True        -0.000919
                       2010-01-05 False       -0.039352
                                  True        -0.037940
                       2010-01-06 False        0.024594
...                                                 ...
GEN_EQ0140919900001000 2011-01-14 True         0.002274
                       2011-01-18 False        0.003606
                                  True         0.003268
                       2011-01-19 False        0.000269
                                  True        -0.000118

[1258000 rows x 1 columns]

In [7]:
day_sep.xs("BBG000B9WH86", level=0)

CumReturnResid
2010-01-04 False       -0.002424
           True        -0.000919
2010-01-05 False       -0.039352
           True        -0.037940
2010-01-06 False        0.024594
...                          ...
2014-12-29 True        -0.005423
2014-12-30 False       -0.005258
           True        -0.005912
2014-12-31 False        0.004876
           True         0.004100

[2516 rows x 1 columns]

In [8]:
R1530_1600 = (-0.000919 +1)/(-0.002424+1) - 1
R1530_1600

0.0015086569845303988

In [9]:
(R1530_1600 + 1) * (-0.039352 + 1) - 1

-0.037902711685124935

In [10]:
adj_end = day_sep.add(1).groupby(level=[0, 1]).pct_change().dropna().droplevel(2)
adj_end

CumReturnResid
Id                                               
BBG000B9WH86           2010-01-04        0.001509
                       2010-01-05        0.001469
                       2010-01-06       -0.003250
                       2010-01-07       -0.002465
                       2010-01-08        0.003632
...                                           ...
GEN_EQ0140919900001000 2011-01-12       -0.001521
                       2011-01-13        0.001727
                       2011-01-14       -0.001101
                       2011-01-18       -0.000336
                       2011-01-19       -0.000387

[626431 rows x 1 columns]

In [11]:
begin = day_sep.xs(False, level=2).shift(-1)
begin

CumReturnResid
Id                                               
BBG000B9WH86           2010-01-04       -0.039352
                       2010-01-05        0.024594
                       2010-01-06       -0.016296
                       2010-01-07       -0.006120
                       2010-01-08        0.021024
...                                           ...
GEN_EQ0140919900001000 2011-01-12        0.006214
                       2011-01-13        0.003378
                       2011-01-14        0.003606
                       2011-01-18        0.000269
                       2011-01-19             NaN

[629000 rows x 1 columns]

In [12]:
target = adj_end.merge(begin, how="inner", left_index=True, right_index=True).stack().add(1).groupby(level=[0, 1]).prod().sub(1).to_frame()

In [13]:
target

0
Id                                         
BBG000B9WH86           2010-01-04 -0.037902
                       2010-01-05  0.026100
                       2010-01-06 -0.019493
                       2010-01-07 -0.008569
                       2010-01-08  0.024732
...                                     ...
GEN_EQ0140919900001000 2011-01-12  0.004684
                       2011-01-13  0.005111
                       2011-01-14  0.002501
                       2011-01-18 -0.000067
                       2011-01-19 -0.000387

[626431 rows x 1 columns]

In [47]:
target_2 = Preprocessor().create_target(raw_df)

In [48]:
target_2

0
Id                                         
BBG000B9WH86           2010-01-04 -0.037902
                       2010-01-05  0.026100
                       2010-01-06 -0.019493
                       2010-01-07 -0.008569
                       2010-01-08  0.024732
...                                     ...
GEN_EQ0140919900001000 2011-01-12  0.004684
                       2011-01-13  0.005111
                       2011-01-14  0.002501
                       2011-01-18 -0.000067
                       2011-01-19 -0.000387

[626431 rows x 1 columns]

In [51]:
raw_df2 = DataHandler().get(end="2011-01-01")

In [72]:
ouioui = raw_df2[["Id", "CumVolume", "Volume"]].groupby(["Id", raw_df2.index.date]).last()

In [83]:
raw_df2[(raw_df2.index.time == dt.time(16, 0)) | (raw_df2.index.time == dt.time(9, 45))].groupby("Id").shift(-1)

,CumReturnResid,CumVolume,EST_VOL,MDV_63,Volume
2010-01-04 09:45:00,0.007571,684807.0,0.13074,43832748.0,955120.0
2010-01-04 09:45:00,-0.015219,2280167.0,0.31610,44092964.0,2566745.0
2010-01-04 09:45:00,-0.006909,754541.0,0.11253,75886260.0,812905.0
2010-01-04 09:45:00,0.010873,1088597.0,0.18568,44133930.0,1108030.0
2010-01-04 09:45:00,0.002427,3220319.0,0.11252,44405412.0,3500468.0
...,...,...,...,...,...
2010-12-31 16:00:00,NaN,NaN,NaN,NaN,NaN
2010-12-31 16:00:00,NaN,NaN,NaN,NaN,NaN
2010-12-31 16:00:00,NaN,NaN,NaN,NaN,NaN
2010-12-31 16:00:00,NaN,NaN,NaN,NaN,NaN


In [ ]:
raw_df2

In [63]:
grouped = raw_df2.groupby(["Id"])

In [64]:
for group in grouped:
    name, test_df = group
    break

/var/folders/9w/vls_pj350g3b62pyz_4t5hp80000gn/T/ipykernel_60186/2545258494.py:1: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for group in grouped:


In [65]:
test_df

,Id,CumReturnResid,CumVolume,EST_VOL,MDV_63,Volume
2010-01-04 09:45:00,BBG000B9WH86,-0.003154,2030006,0.17017,392090530.0,25872368.0
2010-01-04 10:00:00,BBG000B9WH86,-0.002442,3955351,0.17017,392090530.0,25872368.0
2010-01-04 10:15:00,BBG000B9WH86,0.006524,6260704,0.17017,392090530.0,25872368.0
2010-01-04 10:30:00,BBG000B9WH86,0.009743,8236062,0.17017,392090530.0,25872368.0
2010-01-04 10:45:00,BBG000B9WH86,0.005682,9894913,0.17017,392090530.0,25872368.0
...,...,...,...,...,...,...
2010-12-31 15:00:00,BBG000B9WH86,0.010447,9782547,0.13451,304217120.0,14348370.0
2010-12-31 15:15:00,BBG000B9WH86,0.012216,9937005,0.13451,304217120.0,14348370.0
2010-12-31 15:30:00,BBG000B9WH86,0.011395,10184850,0.13451,304217120.0,14348370.0
2010-12-31 15:45:00,BBG000B9WH86,0.015216,11369127,0.13451,304217120.0,14348370.0


In [62]:
grouped.apply()

,Id,CumReturnResid,CumVolume,EST_VOL,MDV_63,Volume
2010-01-04 09:45:00,BBG000B9WH86,-0.003154,2030006,0.17017,392090530.0,25872368.0
2010-01-04 10:00:00,BBG000B9WH86,-0.002442,3955351,0.17017,392090530.0,25872368.0
2010-01-04 10:15:00,BBG000B9WH86,0.006524,6260704,0.17017,392090530.0,25872368.0
2010-01-04 10:30:00,BBG000B9WH86,0.009743,8236062,0.17017,392090530.0,25872368.0
2010-01-04 10:45:00,BBG000B9WH86,0.005682,9894913,0.17017,392090530.0,25872368.0
2010-01-04 11:00:00,BBG000B9WH86,0.003446,11361842,0.17017,392090530.0,25872368.0
2010-01-04 11:15:00,BBG000B9WH86,-0.000290,12414756,0.17017,392090530.0,25872368.0
2010-01-04 11:30:00,BBG000B9WH86,0.002001,13541543,0.17017,392090530.0,25872368.0
2010-01-04 11:45:00,BBG000B9WH86,0.002359,14262978,0.17017,392090530.0,25872368.0
2010-01-04 12:00:00,BBG000B9WH86,0.001421,14787706,0.17017,392090530.0,25872368.0
